In [1]:
# chapter 4-2 クローラー作成にあたっての注意

In [2]:
# robots.txtによるクローラーへの指示

In [3]:
# robots.textのパース

In [4]:
import urllib.robotparser

In [5]:
rp = urllib.robotparser.RobotFileParser()

In [6]:
rp.set_url('http://gihyo.jp/robots.txt') # set_url()でrobots.txtのURLを設定

In [7]:
rp.read() # read()でrobots.txtを読み込む

In [14]:
# can_fetch()の第一引数にUser-Agentの文字列を、第2引数に対象のURLを指定すると、
# そのURLのクロールが許可されているかどうかを取得できる
rp.can_fetch('my_bot', 'http://gihyo.jp/')

False

In [15]:
# HTTP通信におけるエラーへの対処法

In [16]:
# Pythonによるエラー処理

In [54]:
%%writefile error_handling.py

# Pythonによるエラー処理

import time
import requests

TEMPORARY_ERROR_CODES = (400, 500, 502, 503, 504) # 一時的なエラーを表すステータスコード


def main():
    """
    メインとなる処理
    """
    response = fetch('http://httpbin.org/status/200,404,503')
    if 200 <= response.status_code < 300:
        print('Success!')
    else:
        print('Error!')
        
        
def fetch(url):
    """
    指定したURLを取得してResponseオブジェクトを返す
    一時的なエラーが起きた場合は、最大3回リトライする
    """
    max_retries = 3 # 最大で3回リトライする
    retries = 0 # 現在のリトライ回数を示す変数
    while True:
        try:
            print('Retrieving {0}...'.format(url))
            response = requests.get(url)
            print('Status: {0}'.format(response.status_code))
            if response.status_code not in TEMPORARY_ERROR_CODES:
                return response # 一時的なエラーでなければresponseを返す
            
        except requests.exceptions.RequestException as ex:
            # ネットワークレベルのエラー(RequestException)の場合はリトライする
            print('Exception occured: {0}'.format(ex))
            
        retries += 1
        if retries >= max_retries:
            raise Exception('Too many retries.') # リトライ回数の上限を超えた場合は例外を発生させる
                    
        wait = 2**(retries -1) # 指数関数的なリトライ間隔を求める
        print('Waiting {0} seconds...'.format(wait))
        time.sleep(wait) # ウェイトを取る
                

if __name__ == '__main__':
    main()

Overwriting error_handling.py


In [51]:
!python error_handling.py

Retrieving http://httpbin.org/status/200,404,503...
Status: 503
Waiting 1 seconds...
Retrieving http://httpbin.org/status/200,404,503...
Status: 404
Error!


In [30]:
# retryingライブラリを使用して記述を簡潔化

In [53]:
%%writefile error_handling_with_retrying.py

# retryingライブラリを使用して記述を簡潔化

import requests
from retrying import retry

TEMPORARY_ERROR_CODES = (400, 500, 502, 503, 504) # 一時的なエラーを表すステータスコード


def main():
    """
    メインとなる処理
    """
    response = fetch('http://httpbin.org/status/200,404,503')
    if 200 <= response.status_code < 300:
        print('Success!')
    else:
        print('Error!')
        
        
# stop_max_attempt_numberは最大リトライ関数を指定する
# wait_exponential_multiplierは指数関数的なウェイトを取る場合の初回のウェイトをミリ秒単位で指定する
@retry(stop_max_attempt_number=3, wait_exponential_multiplier=1000)
def fetch(url):
    """
    指定したURLを取得してResponseオブジェクトを返す
    一時的なエラーが起きた場合は、最大3回リトライする
    """
    print('Retrieving{0}...'.format(url))
    response = requests.get(url)
    print('Status: {0}'.format(response.status_code))
    if response.status_code not in TEMPORARY_ERROR_CODES:
        return response # 一時的なエラーでなければresponseを返す
    
    # 一時的なエラーの場合は例外を発生させてリトライする
    raise Exception('Temporary Error: {0}'.format(response.status_code))
    
    
if __name__ == '__main__':
    main()

Overwriting error_handling_with_retrying.py


In [41]:
!python error_handling_with_retrying.py

Retrievinghttp://httpbin.org/status/200,404,503...
Status: 503
Retrievinghttp://httpbin.org/status/200,404,503...
Status: 404
Error!


In [52]:
# chapter 4-3 繰り返しの実行を前提とした設計

In [ ]:
# 更新されたデータだけを取得する

In [ ]:
# CacheControlを使ってキャッシュを処理する

In [68]:
%%writefile request_with_cache.py

# CacheControlを使ってキャッシュを処理する

import requests
from cachecontrol import CacheControl

session = requests.session()
cached_session = CacheControl(session) # sessionをラップしたcached_sessionを作る

# 1回目はキャッシュがないのでサーバーから取得しキャッシュする
response = cached_session.get('http://docs.python.org/3/')
print(response.from_cache) # False

# 2回目以降はETagとLast-Modifiedの値を使って更新されているかを確認する
# 更新されていない場合のコンテンツはキャッシュから取得するので高速に処理できる
response = cached_session.get('http://docs.python.org/3/')
print(response.from_cache) # True

Overwriting request_with_cache.py


In [69]:
!python request_with_cache.py

False
True


In [1]:
# chapter 4-4 クロール先の変化に対応する

In [2]:
# 変化を検知する

In [3]:
# 正規表現で価格として正しいかチェックする

In [13]:
%%writefile validate_with_re.py

# 正規表現で価格として正しいかチェックする

import re

value = '3,000'
if not re.search(r'^[0-9,]+$', value): # 数字とカンマのみを含む正規表現にマッチするかチェックする
    raise ValueError('Invalid price') # 正しい値でない場合は例外を発生させる

Overwriting validate_with_re.py


In [14]:
!python validate_with_re.py

In [15]:
# ライブラリを使用してスキーマを定義する

In [16]:
# Voluptuousによるバリデーション

In [24]:
%%writefile validate_with_voluptuous.py

# Voluptuousによるバリデーション

from voluptuous import Schema, Match

# 次の4つのルールを持つスキーマを定義する
schema = Schema({ # ルール1：オブジェクトはdictである
    'name': str, # ルール2：nameの値は文字列である
    'price': Match(r'^[0-9,]+$'), # ルール3：priceの値は正規表現にマッチする
}, required=True) # ルール4：dictのキーは必須である

# Schemaオブジェクトを関数として呼び出すと、引数のオブジェクトを対象にバリデーションを行う
schema({
    'name': 'ぶどう',
    'price': '3,000',
}) #スキーマに適合するので例外は発生しない

schema({
    'name': None,
    'price': '3,000',
}) # スキーマに適合しないので、例外MultipleInvalidが発生する

Overwriting validate_with_voluptuous.py


In [25]:
!python validate_with_voluptuous.py

Traceback (most recent call last):
  File "validate_with_voluptuous.py", line 20, in <module>
    'price': '3,000',
  File "/Users/Really/.pyenv/versions/anaconda3-4.3.1/lib/python3.6/site-packages/voluptuous/schema_builder.py", line 221, in __call__
    return self._compiled([], data)
  File "/Users/Really/.pyenv/versions/anaconda3-4.3.1/lib/python3.6/site-packages/voluptuous/schema_builder.py", line 538, in validate_dict
    return base_validate(path, iteritems(data), out)
  File "/Users/Really/.pyenv/versions/anaconda3-4.3.1/lib/python3.6/site-packages/voluptuous/schema_builder.py", line 370, in validate_mapping
    raise er.MultipleInvalid(errors)
voluptuous.error.MultipleInvalid: expected str for dictionary value @ data['name']


In [22]:
# 変化を通知する

In [23]:
# Pythonからメールを送信する

In [36]:
%%writefile send_email.py

# メールを送信する(Gmailを使用)

import smtplib
from email.mime.text import MIMEText
from email.header import Header

msg = MIMEText('メールの本文です') # MIMETextオブジェクトでメッセージを組み立てる
msg['Subject'] = Header('メールの件名', 'utf-8') # 件名に日本語を含める場合はHeaderオブジェクトを使う
msg['From'] = 'me@gmail.com' # 差出人のメールアドレス
msg['To'] = 'you@gmail.com' # 送信先のメールアドレス

with smtplib.SMTP_SSL('smtp.gmail.com') as smtp: # SMTP()の第1引数にSMTPサーバーのホスト名を指定する
    # Googleアカウントのユーザー名とパスワードを指定してログインする
    # 2段階認証を設定している場合は、アプリパスワードを生成して使用する
    smtp.login('me@gmail.com', 'パスワード')
    smtp.send_message(msg) # send_message()メソッドでメールを送信する

Overwriting send_email.py


In [35]:
!python send_email.py